In [1]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = "f6fba7125b754c0d93f2fce1da3a43a3"
RESOURCE_ENDPOINT = "https://ciaulima.openai.azure.com/"
print(API_KEY)
print(RESOURCE_ENDPOINT)

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

f6fba7125b754c0d93f2fce1da3a43a3
https://ciaulima.openai.azure.com/
{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "ada002cia",
      "status": "succeeded",
      "created_at": 1693643909,
      "updated_at": 1693643909,
      "object": "deployment"
    }
  ],
  "object": "list"
}


In [2]:
#Un error que se puede presentar es por el encoding del csv, en caso suceda ಠ_ಥ
#Hay que correr el archivo en txt y guardarlo como csv con encoding utf-8 
df=pd.read_csv(os.path.join(os.getcwd(),'cia_data.csv')) # This assumes that you have placed the ceocom_dataset.csv in the same directory you are running Jupyter Notebooks
df

,id,bill_id,text,summary,title,text_len,sum_len
0,0,1,!Hola! !Bienvenid@ al chat del Círculo de Inte...,"Hola, Buenos dias, Buenas tardes, Holi, Bienve...","Hola, hola, Saludos, Buenos dias, Buenas tarde...",8494,321
1,1,2,Nuestro objetivo es fomentar la colaboración y...,Objetivo del Cia,Objetivo del Cia,6522,1424
2,2,3,Sobre el área de investigación y realización d...,"Investigacion, papers",Area de Investigacion,6154,463
3,3,4,Sobre el área de Marketing:\n📣💼 En el Círculo ...,Area de Marketing,Area de Marketing,19853,1400
4,4,5,Sobre el área de Formación Académica:\n🎓📚 Adem...,"Formacion Academica, talleres",Area de Formacion Academica,6273,278
5,5,6,La tribu de proyectos estudia y hace proyectos :D,Tribu de Proyectos,Tribu de Proyectos,11691,114
6,6,7,La tribu de algoritmos estudia y trabaja con a...,Tribu de Algoritmos,Tribu de Algoritmos,5328,379
7,7,8,La tribu de modelos estudia y hace proyectos a...,Tribu de Modelos,Tribu de Modelos,16668,1525
8,8,9,La tribu de Herramientas e Impacto hace public...,Tribu de Herramientas e Impacto,Tribu de Herramientas e Impacto,15352,2151
9,9,10,Hasta luego chikistrikis!,"Despedida, Chau, Hasta Luego, Bye","Despedida, Chau, Hasta Luego, Bye",5633,894


In [3]:
df_bills = df[['text', 'summary', 'title']]
df_bills

,text,summary,title
0,!Hola! !Bienvenid@ al chat del Círculo de Inte...,"Hola, Buenos dias, Buenas tardes, Holi, Bienve...","Hola, hola, Saludos, Buenos dias, Buenas tarde..."
1,Nuestro objetivo es fomentar la colaboración y...,Objetivo del Cia,Objetivo del Cia
2,Sobre el área de investigación y realización d...,"Investigacion, papers",Area de Investigacion
3,Sobre el área de Marketing:\n📣💼 En el Círculo ...,Area de Marketing,Area de Marketing
4,Sobre el área de Formación Académica:\n🎓📚 Adem...,"Formacion Academica, talleres",Area de Formacion Academica
5,La tribu de proyectos estudia y hace proyectos :D,Tribu de Proyectos,Tribu de Proyectos
6,La tribu de algoritmos estudia y trabaja con a...,Tribu de Algoritmos,Tribu de Algoritmos
7,La tribu de modelos estudia y hace proyectos a...,Tribu de Modelos,Tribu de Modelos
8,La tribu de Herramientas e Impacto hace public...,Tribu de Herramientas e Impacto,Tribu de Herramientas e Impacto
9,Hasta luego chikistrikis!,"Despedida, Chau, Hasta Luego, Bye","Despedida, Chau, Hasta Luego, Bye"


In [4]:
#El texto tiene \n por todas partes, necesita normalizarse ʕ•ᴥ•ʔ
pd.options.mode.chained_assignment = None #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#evaluation-order-matters

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_bills['text']= df_bills["text"].apply(lambda x : normalize_text(x))

In [5]:
df_bills

,text,summary,title
0,!Hola! !Bienvenid@ al chat del Círculo de Inte...,"Hola, Buenos dias, Buenas tardes, Holi, Bienve...","Hola, hola, Saludos, Buenos dias, Buenas tarde..."
1,Nuestro objetivo es fomentar la colaboración y...,Objetivo del Cia,Objetivo del Cia
2,Sobre el área de investigación y realización d...,"Investigacion, papers",Area de Investigacion
3,Sobre el área de Marketing: 📣💼 En el Círculo d...,Area de Marketing,Area de Marketing
4,Sobre el área de Formación Académica: 🎓📚 Ademá...,"Formacion Academica, talleres",Area de Formacion Academica
5,La tribu de proyectos estudia y hace proyectos :D,Tribu de Proyectos,Tribu de Proyectos
6,La tribu de algoritmos estudia y trabaja con a...,Tribu de Algoritmos,Tribu de Algoritmos
7,La tribu de modelos estudia y hace proyectos a...,Tribu de Modelos,Tribu de Modelos
8,La tribu de Herramientas e Impacto hace public...,Tribu de Herramientas e Impacto,Tribu de Herramientas e Impacto
9,Hasta luego chikistrikis!,"Despedida, Chau, Hasta Luego, Bye","Despedida, Chau, Hasta Luego, Bye"


In [6]:
#Calculamos cantidad de Tokens y eliminamos filas que excedan límite de tokens ʕ•ᴥ•ʔ
tokenizer = tiktoken.get_encoding("cl100k_base")
df_bills['n_tokens'] = df_bills["text"].apply(lambda x: len(tokenizer.encode(x)))
df_bills = df_bills[df_bills.n_tokens<8192]
len(df_bills)

10

In [7]:
df_bills

,text,summary,title,n_tokens
0,!Hola! !Bienvenid@ al chat del Círculo de Inte...,"Hola, Buenos dias, Buenas tardes, Holi, Bienve...","Hola, hola, Saludos, Buenos dias, Buenas tarde...",270
1,Nuestro objetivo es fomentar la colaboración y...,Objetivo del Cia,Objetivo del Cia,83
2,Sobre el área de investigación y realización d...,"Investigacion, papers",Area de Investigacion,95
3,Sobre el área de Marketing: 📣💼 En el Círculo d...,Area de Marketing,Area de Marketing,100
4,Sobre el área de Formación Académica: 🎓📚 Ademá...,"Formacion Academica, talleres",Area de Formacion Academica,78
5,La tribu de proyectos estudia y hace proyectos :D,Tribu de Proyectos,Tribu de Proyectos,12
6,La tribu de algoritmos estudia y trabaja con a...,Tribu de Algoritmos,Tribu de Algoritmos,18
7,La tribu de modelos estudia y hace proyectos a...,Tribu de Modelos,Tribu de Modelos,18
8,La tribu de Herramientas e Impacto hace public...,Tribu de Herramientas e Impacto,Tribu de Herramientas e Impacto,22
9,Hasta luego chikistrikis!,"Despedida, Chau, Hasta Luego, Bye","Despedida, Chau, Hasta Luego, Bye",9


In [8]:
#Vemos precisamente como son separados los tokens en la fila seleccionada
sample_encode = tokenizer.encode(df_bills.text[0]) #Número de fila
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'!',
 b'Hola',
 b'!',
 b' !',
 b'Bien',
 b'ven',
 b'id',
 b'@',
 b' al',
 b' chat',
 b' del',
 b' C',
 b'\xc3\xad',
 b'rc',
 b'ulo',
 b' de',
 b' Intel',
 b'ig',
 b'encia',
 b' Artificial',
 b' (',
 b'C',
 b'IA',
 b')!',
 b' \xf0\x9f\x91',
 b'\x8b',
 b' El',
 b' CIA',
 b' se',
 b' ded',
 b'ica',
 b' a',
 b' la',
 b' investigaci\xc3\xb3n',
 b',',
 b' desarrollo',
 b' de',
 b' proyectos',
 b' y',
 b' la',
 b' producci\xc3\xb3n',
 b' de',
 b' papers',
 b' en',
 b' el',
 b' campo',
 b' de',
 b' la',
 b' Intel',
 b'ig',
 b'encia',
 b' Artificial',
 b'.',
 b' \xf0\x9f',
 b'\xa4',
 b'\x96',
 b'\xf0\x9f',
 b'\x94',
 b'\xac',
 b' N',
 b'uest',
 b'ro',
 b' c',
 b'\xc3\xad',
 b'rc',
 b'ulo',
 b' se',
 b' comp',
 b'one',
 b' de',
 b' diversas',
 b' \xc3\xa1reas',
 b',',
 b' incl',
 b'uy',
 b'endo',
 b' el',
 b' area',
 b' de',
 b' investigaci\xc3\xb3n',
 b' y',
 b' papers',
 b',',
 b' el',
 b' area',
 b' de',
 b' marketing',
 b' y',
 b' el',
 b' area',
 b' de',
 b' desarrollo',
 b' de',
 b' clas

In [9]:
len(decode) #Coincide con la columna n_tokens

270

In [10]:
df_bills['ada_v2'] = df_bills["text"].apply(lambda x : get_embedding(x, engine = 'ada002cia'))

In [11]:
df_bills

,text,summary,title,n_tokens,ada_v2
0,!Hola! !Bienvenid@ al chat del Círculo de Inte...,"Hola, Buenos dias, Buenas tardes, Holi, Bienve...","Hola, hola, Saludos, Buenos dias, Buenas tarde...",270,"[-0.02597006782889366, -0.008009089156985283, ..."
1,Nuestro objetivo es fomentar la colaboración y...,Objetivo del Cia,Objetivo del Cia,83,"[-0.020858202129602432, -0.012345310300588608,..."
2,Sobre el área de investigación y realización d...,"Investigacion, papers",Area de Investigacion,95,"[-0.016345854848623276, -0.006409612949937582,..."
3,Sobre el área de Marketing: 📣💼 En el Círculo d...,Area de Marketing,Area de Marketing,100,"[-0.01971987634897232, -0.009440229274332523, ..."
4,Sobre el área de Formación Académica: 🎓📚 Ademá...,"Formacion Academica, talleres",Area de Formacion Academica,78,"[-0.01594971865415573, -0.028691085055470467, ..."
5,La tribu de proyectos estudia y hace proyectos :D,Tribu de Proyectos,Tribu de Proyectos,12,"[-0.014809500426054, -0.016615860164165497, 0...."
6,La tribu de algoritmos estudia y trabaja con a...,Tribu de Algoritmos,Tribu de Algoritmos,18,"[-0.012540304102003574, -0.004291333723813295,..."
7,La tribu de modelos estudia y hace proyectos a...,Tribu de Modelos,Tribu de Modelos,18,"[-0.0307281706482172, 0.003042606869712472, 0...."
8,La tribu de Herramientas e Impacto hace public...,Tribu de Herramientas e Impacto,Tribu de Herramientas e Impacto,22,"[-0.033724747598171234, -0.022617634385824203,..."
9,Hasta luego chikistrikis!,"Despedida, Chau, Hasta Luego, Bye","Despedida, Chau, Hasta Luego, Bye",9,"[-0.0054408409632742405, -0.011277618817985058..."


In [12]:
# OPCIONALMENTE DESCARGAMOS LA TABLA CON LOS EMBEDDINGS EN FORMATO CSV UTF-8 CON ESTE CODE SNIPPET ʕ•ᴥ•ʔ 
# TRAS LA DESCARGA SE USARA EL ARCHIVO CSV YA NORMALIZADO PARA EVITAR LA INNECESARIA LA NORMALIZACION DEL MODELO A CADA RATO Y AHORRAR TIEMPO

from IPython.display import FileLink, display

# Assuming df_bills is your dataframe

# Convert dataframe to CSV
csv_data = df_bills.to_csv(index=False)

# Save the CSV file with UTF-8 encoding
csv_file = 'bills.csv'
with open(csv_file, 'w', encoding='utf-8') as f:
    f.write(csv_data)

# Create a download link
display(FileLink(csv_file))


c:\Users\Nachito\Desktop\Bitbucket\ceocom-azureopenai-poc\azure_openai_doc&jupyntbook\Embeddings\Cia_Embedding\bills.csv

In [13]:
df = pd.read_csv('bills.csv')
df

,text,summary,title,n_tokens,ada_v2
0,!Hola! !Bienvenid@ al chat del Círculo de Inte...,"Hola, Buenos dias, Buenas tardes, Holi, Bienve...","Hola, hola, Saludos, Buenos dias, Buenas tarde...",270,"[-0.02597006782889366, -0.008009089156985283, ..."
1,Nuestro objetivo es fomentar la colaboración y...,Objetivo del Cia,Objetivo del Cia,83,"[-0.020858202129602432, -0.012345310300588608,..."
2,Sobre el área de investigación y realización d...,"Investigacion, papers",Area de Investigacion,95,"[-0.016345854848623276, -0.006409612949937582,..."
3,Sobre el área de Marketing: 📣💼 En el Círculo d...,Area de Marketing,Area de Marketing,100,"[-0.01971987634897232, -0.009440229274332523, ..."
4,Sobre el área de Formación Académica: 🎓📚 Ademá...,"Formacion Academica, talleres",Area de Formacion Academica,78,"[-0.01594971865415573, -0.028691085055470467, ..."
5,La tribu de proyectos estudia y hace proyectos :D,Tribu de Proyectos,Tribu de Proyectos,12,"[-0.014809500426054, -0.016615860164165497, 0...."
6,La tribu de algoritmos estudia y trabaja con a...,Tribu de Algoritmos,Tribu de Algoritmos,18,"[-0.012540304102003574, -0.004291333723813295,..."
7,La tribu de modelos estudia y hace proyectos a...,Tribu de Modelos,Tribu de Modelos,18,"[-0.0307281706482172, 0.003042606869712472, 0...."
8,La tribu de Herramientas e Impacto hace public...,Tribu de Herramientas e Impacto,Tribu de Herramientas e Impacto,22,"[-0.033724747598171234, -0.022617634385824203,..."
9,Hasta luego chikistrikis!,"Despedida, Chau, Hasta Luego, Bye","Despedida, Chau, Hasta Luego, Bye",9,"[-0.0054408409632742405, -0.011277618817985058..."


In [15]:
# this code snippet search through the reviews for a specific answer
# df is the dataframe with the embeddings so it means it can be replacede with a csvfile or a blob


#IF 'to_print' is set to True, the function will also print the obtained ROWS, else it won't
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        #engine="text-embedding-ada-002" engine debe ser identico al deployment name del modelo
		engine="ada002cia"
    )
    df["similarities"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

user_query = input("Ingresar el texto: ")
res = search_docs(df_bills, user_query, top_n=3) #Aqui va la pregunta a realizar
first_row = res.iloc[0]  # Get the first row of the result
text_value = first_row['text']  # Fetch the value of the 'text' column
print('Respuesta obtenida por parte del modelo:')
print(text_value)  # Print the value of the 'text' column

,text,summary,title,n_tokens,ada_v2,similarities
0,!Hola! !Bienvenid@ al chat del Círculo de Inte...,"Hola, Buenos dias, Buenas tardes, Holi, Bienve...","Hola, hola, Saludos, Buenos dias, Buenas tarde...",270,"[-0.02597006782889366, -0.008009089156985283, ...",0.855134
3,Sobre el área de Marketing: 📣💼 En el Círculo d...,Area de Marketing,Area de Marketing,100,"[-0.01971987634897232, -0.009440229274332523, ...",0.848236
2,Sobre el área de investigación y realización d...,"Investigacion, papers",Area de Investigacion,95,"[-0.016345854848623276, -0.006409612949937582,...",0.843239


Respuesta obtenida por parte del modelo:
!Hola! !Bienvenid@ al chat del Círculo de Inteligencia Artificial (CIA)! 👋 El CIA se dedica a la investigación, desarrollo de proyectos y la producción de papers en el campo de la Inteligencia Artificial. 🤖🔬 Nuestro círculo se compone de diversas áreas, incluyendo el area de investigación y papers, el area de marketing y el area de desarrollo de clases, así como cuatro tribus enfocadas en algoritmos, modelos de IA, proyectos de IA y herramientas e impacto de IA. 🤝😊 No dudes en hacer cualquier pregunta que tengas sobre nuestras áreas, tribus o proyectos! Estamos aquí para ayudarte y compartir nuestro entusiasmo por la Inteligencia Artificial. 🌟🤖💡 ¡Esperamos poder brindarte información valiosa y resolver todas tus dudas! 💪😊 Para ser concisos, que es lo que buscas pe chat@? 💀 - Objetivo del CIA - Area de Investigacion - Area de Marketing - Area de Formacion Academica - Tribu de Algoritmos - Tribu de Proyectos - Tribu de Modelos - Tribu de Herramien